# Example 2 - Database Management with MatGraphDB

In this tutorial, we'll explore basic database management operations—adding, reading, updating, and deleting materials—using the `MatGraphDB` class and its `MaterialDatabaseManager` instance. The `MaterialDatabaseManager` serves as a Data Access Layer, wrapping around the Atomic Simulation Environment (ASE) database while adding custom functionalities specific to our package.

**Table of Contents**

1. [Introduction](#Introduction)
2. [Database Details](#Database-Details)
3. [Setup](#Setup)
4. [Adding Materials](#Adding-Materials)
    - [Understanding Data Storage: `data` vs. Keyword Arguments](#understanding-data-storage-data-vs-keyword-arguments)
    - [Handling Large Numbers of Key-Value Pairs with External Tables](#handling-large-numbers-of-key-value-pairs-with-external-tables)
5. [Reading Materials](#Reading-Materials)
6. [Updating Materials](#Updating-Materials)
7. [Deleting Materials](#Deleting-Materials)
8. [Conclusion](#Conclusion)

---

## Introduction

`MatGraphDB` is a database interface designed for managing materials data efficiently. It leverages the ASE database, allowing users to store and retrieve atomic structures and associated data seamlessly.

- **`MatGraphDB`**: The main class that users interact with. It contains an instance variable `db_manager`, which is an initialized `MaterialDatabaseManager`.
- **`MaterialDatabaseManager`**: Acts as the Data Access Layer, providing methods for basic database operations like adding, reading, updating, and deleting materials.

For more detailed information about the underlying ASE database functionalities, you can refer to the [ASE Database Documentation](https://wiki.fysik.dtu.dk/ase/ase/db/db.html).

---

## Database Details

This information comes from the ASE database documentation. [ASE Database Documentation](https://wiki.fysik.dtu.dk/ase/ase/db/db.html).

### Type of Database

ASE support multiple database types: JSON, SQLite3, PostgreSQL, MySQL, MariaDB, and SQLite3. Currently, MatGraphDB only supports SQLite3.

### Row Information

Every row in the database contains:

- all the information stored in the `ase.Atoms` object (positions, atomic numbers, …)
- calculator name and parameters (if a calculator is present)
- already calculated properties such as energy and forces (if a calculator is present)
- key-value pairs (for finding the calculation again)
- an integer ID (unique for each database) starting with 1 and always increasing for each new row
- a unique ID which is a 128 bit random number which should be globally unique (at least in the lifetime of our universe)
- constraints (if present)
- user-name
- creation and modification time

### Row Special Names

These names are special:

| **Name**      | **Description**                                                                       |
|---------------|---------------------------------------------------------------------------------------|
| `id`          | Integer identifier                                                                    |
| `natoms`      | Number of atoms                                                                       |
| `pbc`         | Periodic boundary conditions                                                          |
| `formula`     | Formula                                                                               |
| `energy`      | Potential energy                                                                      |
| `charge`      | Total charge                                                                          |
| `magmom`      | Total magnetic moment                                                                 |
| `calculator`  | Name of calculator                                                                    |
| `user`        | Who did it                                                                            |
| `age`         | Age of calculation (use s, m, h, d, w, M, and y for second, minute, hour, day, week, month, and year respectively) |


### Description of a row
The first 9 keys (from “id” to “positions”) are always present — the rest may be there:

| **Key**                | **Description**                        | **Datatype**          | **Shape**       |
|------------------------|----------------------------------------|-----------------------|-----------------|
| `id`                   | Local database id                      | int                   | Unique          |
| `unique_id`            | Globally unique hexadecimal id          | str                   | Unique          |
| `ctime`                | Creation time                          | float                 | Unique          |
| `mtime`                | Modification time                      | float                 | Unique          |
| `user`                 | User name                              | str                   | Unique          |
| `numbers`              | Atomic numbers                         | int                   | (N,)            |
| `pbc`                  | Periodic boundary condition flags      | bool                  | (3,)            |
| `cell`                 | Unit cell                              | float                 | (3, 3)          |
| `positions`            | Atomic positions                       | float                 | (N, 3)          |
| `initial_magmoms`      | Initial atomic magnetic moments        | float                 | (N,)            |
| `initial_charges`      | Initial atomic charges                 | float                 | (N,)            |
| `masses`               | Atomic masses                          | float                 | (N,)            |
| `tags`                 | Tags                                   | int                   | (N,)            |
| `momenta`              | Atomic momenta                         | float                 | (N, 3)          |
| `constraints`          | Constraints                            | list of dict          | Varies          |
| `energy`               | Total energy                           | float                 | Unique          |
| `forces`               | Atomic forces                          | float                 | (N, 3)          |
| `stress`               | Stress tensor                          | float                 | (6,)            |
| `dipole`               | Electrical dipole                      | float                 | (3,)            |
| `charges`              | Atomic charges                         | float                 | (N,)            |
| `magmom`               | Magnetic moment                        | float                 | Unique          |
| `magmoms`              | Atomic magnetic moments                | float                 | (N,)            |
| `calculator`           | Calculator name                        | str                   | Unique          |
| `calculator_parameters`| Calculator parameters                  | dict                  | Varies          |


### Key-Value Pairs
Every row also stores key-value pairs in a text field called `key_value_pairs`. This values should be simple data types such as strings, integers, floats, or booleans.

### Data
Every row also stores data in a text filed called `data`. This field is a dictionary that can store any data that is needed for the material. The data types for this can be as complicated that can fit into a serializable dictionary. When the database is queried, the `data` field is automatically converted to a dictionary.

### External Tables
If you need to store a large number of key-value pairs (e.g., thousands of features for a machine learning model), the `key_value_pairs` table can become unwieldy. ASE provides a solution through **external tables**.





## Setup

First, let's set up our environment by importing necessary libraries and initializing the `MatGraphDB` instance.

In [1]:
# Import necessary libraries
import logging
import os
import numpy as np
from matgraphdb import MatGraphDB

# Configure logging
logger = logging.getLogger('matgraphdb')
logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)

logger.addHandler(ch)

### Initialize MatGraphDB

We need to specify the directory where our database will reside. 

In [2]:
# Initialize MatGraphDB
mgdb = MatGraphDB(main_dir=os.path.join('data', 'MatGraphDB_Example'))

2024-09-19 14:33:35,482 - matgraphdb.core - INFO - Initializing MaterialRepositoryHandler.
2024-09-19 14:33:35,484 - matgraphdb.core - DEBUG - Main directory set to data\MatGraphDB_Example
2024-09-19 14:33:35,485 - matgraphdb.core - DEBUG - Calculation directory set to data\MatGraphDB_Example\calculations
2024-09-19 14:33:35,485 - matgraphdb.core - DEBUG - Graph directory set to data\MatGraphDB_Example\graph_database
2024-09-19 14:33:35,486 - matgraphdb.core - DEBUG - Database path set to data\MatGraphDB_Example\materials.db
2024-09-19 14:33:35,487 - matgraphdb.core - DEBUG - Number of cores set to 1
2024-09-19 14:33:35,487 - matgraphdb.data.material_manager - INFO - Initializing MaterialDatabaseManager with database at data\MatGraphDB_Example\materials.db
2024-09-19 14:33:36,142 - matgraphdb.core - DEBUG - MaterialDatabaseManager initialized.
2024-09-19 14:33:36,144 - matgraphdb.data.calc_manager - INFO - Initializing CalculationManager with main directory: data\MatGraphDB_Example\cal

___

## MaterialDatabaseManager

The `MaterialDatabaseManager` class is the main class for interacting with the database. It provides methods for adding, reading, updating, deleting materials  from the database. The `MatGraphDB` has an instance of `MaterialDatabaseManager` that is initialized when the `MatGraphDB` is initialized.

You can access the `MaterialDatabaseManager` instance using the `db_manager` attribute of the `MatGraphDB` class.

In [3]:
print(mgdb.db_manager.db_path)

data\MatGraphDB_Example\materials.db


When the `MaterialDatabaseManager` is initialized, it creates automatically creates a connection to the ASE database. With this you have direct access to the ASE database.

In [4]:
print(mgdb.db_manager.db.__class__)

<class 'ase.db.sqlite.SQLite3Database'>


---

## Adding Materials

You can add materials to the database using the `add_material` method. You can provide the atomic coordinates, species, lattice parameters, and any additional data.

### Example 1: Adding a Material with Structure

In [5]:
# Add a material with atomic structure and additional data
mgdb.db_manager.add_material(
    coords=[[0, 0, 0]],
    species=['Fe'],
    lattice=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    data={'density': 7.87}
)

2024-09-19 14:33:36,205 - matgraphdb.data.material_manager - INFO - Adding a new material.
2024-09-19 14:33:36,206 - matgraphdb.data.material_manager - DEBUG - Processing composition input.
2024-09-19 14:33:36,207 - matgraphdb.data.material_manager - DEBUG - No valid composition information provided.
2024-09-19 14:33:36,207 - matgraphdb.data.material_manager - DEBUG - Processing structure input.
2024-09-19 14:33:36,209 - matgraphdb.data.material_manager - DEBUG - Building ASE Atoms structure from provided coordinates, species, and lattice.
2024-09-19 14:33:36,210 - matgraphdb.data.material_manager - DEBUG - Adding custom data: {'density': 7.87}
2024-09-19 14:33:36,301 - matgraphdb.data.material_manager - INFO - Material added successfully.


### Example 2: Adding Multiple Materials

The most effiecient way to multiple matrerials is to use the `add_many` method. This method accepts a list of dictionaries, where each dictionary are the key_value pairs for a material that `add_materials` expects. This is faster than adding each material individually, it adds all the materials to permanent storage in one transaction rather than one transaction per material.

In [6]:
# Add multiple materials
materials=[]
for density in [7.87, 8.96, 2.70]:
    material_dict={
        'coords': [[0, 0, 0]],
        'species': ['Fe'],
        'lattice': [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
        'data': {'density': density}
    }

mgdb.db_manager.add_many(materials)

2024-09-19 14:33:36,311 - matgraphdb.data.material_manager - INFO - Adding 0 materials to the database.
2024-09-19 14:33:36,313 - matgraphdb.data.material_manager - INFO - All materials added successfully.


### Example 3: Adding a Material with Composition Only

If you don't have the atomic structure but know the composition, you can add a material using the `composition` parameter.

In [7]:
# Add a material with composition only
mgdb.db_manager.add_material(composition='TiO2')

2024-09-19 14:33:36,327 - matgraphdb.data.material_manager - INFO - Adding a new material.
2024-09-19 14:33:36,329 - matgraphdb.data.material_manager - DEBUG - Processing composition input.
2024-09-19 14:33:36,329 - matgraphdb.data.material_manager - DEBUG - Composition provided as string: TiO2
2024-09-19 14:33:36,330 - matgraphdb.data.material_manager - DEBUG - Processing structure input.
2024-09-19 14:33:36,331 - matgraphdb.data.material_manager - DEBUG - No valid structure information provided.
2024-09-19 14:33:36,434 - matgraphdb.data.material_manager - INFO - Material added successfully.


### Understanding Data Storage: `data` vs. Keyword Arguments

When adding materials, you can store additional information using either the `data` parameter or by adding keyword arguments directly to the `add_material` method. It's important to understand the difference between these two approaches.

#### Using the `data` Parameter

- **Purpose**: Store custom data associated with the material. This is more long-term storage.
- **Storage**: Data is stored in the `data` field of the database entry.
- **Searchability**: Not directly searchable using ASE's selection syntax.
- **Usage**:

In [8]:
mgdb.db_manager.add_material(
      coords=np.array([[0, 0, 0]]),
      species=['Fe'],
      lattice=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
      data={'density': 7.87, 'color': 'silver'}
  )

2024-09-19 14:33:36,451 - matgraphdb.data.material_manager - INFO - Adding a new material.
2024-09-19 14:33:36,452 - matgraphdb.data.material_manager - DEBUG - Processing composition input.
2024-09-19 14:33:36,452 - matgraphdb.data.material_manager - DEBUG - No valid composition information provided.
2024-09-19 14:33:36,453 - matgraphdb.data.material_manager - DEBUG - Processing structure input.
2024-09-19 14:33:36,454 - matgraphdb.data.material_manager - DEBUG - Building ASE Atoms structure from provided coordinates, species, and lattice.
2024-09-19 14:33:36,454 - matgraphdb.data.material_manager - DEBUG - Adding custom data: {'density': 7.87, 'color': 'silver'}
2024-09-19 14:33:36,542 - matgraphdb.data.material_manager - INFO - Material added successfully.


#### Using Keyword Arguments

- **Purpose**: Store data that needs to be searchable or indexed.
- **Storage**: Each key-value pair is stored in its own column in the database's `key_value_pairs` table.
- **Searchability**: Directly searchable using ASE's selection syntax.
- **Considerations**:
  - Adding many keyword arguments can lead to a large `key_value_pairs` table, which can impact performance.
- **Usage**:

In [9]:
# Adding key-value pairs as keyword arguments
mgdb.db_manager.add_material(
    coords=np.array([[0, 0, 0]]),
    species=['Fe'],
    lattice=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    is_molecule=True,  # This is a keyword argument
    phase='solid' # This is a keyword argument
)

2024-09-19 14:33:36,559 - matgraphdb.data.material_manager - INFO - Adding a new material.
2024-09-19 14:33:36,560 - matgraphdb.data.material_manager - DEBUG - Processing composition input.
2024-09-19 14:33:36,561 - matgraphdb.data.material_manager - DEBUG - No valid composition information provided.
2024-09-19 14:33:36,562 - matgraphdb.data.material_manager - DEBUG - Processing structure input.
2024-09-19 14:33:36,563 - matgraphdb.data.material_manager - DEBUG - Building ASE Atoms structure from provided coordinates, species, and lattice.
2024-09-19 14:33:36,659 - matgraphdb.data.material_manager - INFO - Material added successfully.


#### Example: Searching Using Keyword Arguments

Since keyword arguments are stored in a searchable manner, you can perform queries based on them. 

The `selection` keyword argument is used to specify the selection criteria for the query. In this example, we are searching for materials where the `is_molecule` field is set to `True`.

Can be:
- an integer id
- a string like ‘key=value’, where ‘=’ can also be one of ‘<=’, ‘<’, ‘>’, ‘>=’ or ‘!=’.
- a string like ‘key’
- comma separated strings like ‘key1<value1,key2=value2,key’
- list of strings or tuples: [(‘charge’, ‘=’, 1)].

In [10]:
# Read materials where 'is_molecule' is True
results = mgdb.db_manager.read(selection='is_molecule=True')

for result in results:
    print(f"ID: {result.id}, Is Molecule: {result.is_molecule}")

2024-09-19 14:33:36,667 - matgraphdb.data.material_manager - DEBUG - Reading materials with selection: is_molecule=True, filters: {}


ID: 4, Is Molecule: True


#### Be Cautious with Keyword Arguments

While keyword arguments are useful for creating searchable indices, be cautious when adding many of them:

- **Database Size**: The `key_value_pairs` table can grow rapidly, impacting performance.
- **Maintenance**: Managing a large number of keys can become complex.

### Handling Large Numbers of Key-Value Pairs with External Tables

If you need to store a large number of key-value pairs (e.g., thousands of features for a machine learning model), the `key_value_pairs` table can become unwieldy. ASE provides a solution through **external tables**.

#### Example: Adding Material with External Table

In [11]:
from ase import Atoms

# Create a large feature dictionary
no_features = 5000
feature_dict = {'feature' + str(i): i for i in range(no_features)}

# Add material with external table 'features'
mgdb.db_manager.add_material(
    coords=np.array([[0, 0, 0]]),
    species=['Fe'],
    lattice=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    external_tables={'features': feature_dict}
)

2024-09-19 14:33:36,686 - matgraphdb.data.material_manager - INFO - Adding a new material.
2024-09-19 14:33:36,688 - matgraphdb.data.material_manager - DEBUG - Processing composition input.
2024-09-19 14:33:36,688 - matgraphdb.data.material_manager - DEBUG - No valid composition information provided.
2024-09-19 14:33:36,689 - matgraphdb.data.material_manager - DEBUG - Processing structure input.
2024-09-19 14:33:36,690 - matgraphdb.data.material_manager - DEBUG - Building ASE Atoms structure from provided coordinates, species, and lattice.
2024-09-19 14:33:36,860 - matgraphdb.data.material_manager - INFO - Material added successfully.


#### Accessing Data in External Tables

In [12]:
# Retrieve the material
results = mgdb.db_manager.read(5)

# Access features from the external table
for result in results:
    feature1 = result['features']['feature1']
    feature4999 = result['features']['feature4999']


print(f"Feature1: {feature1}")
print(f"Feature4999: {feature4999}")

2024-09-19 14:33:36,872 - matgraphdb.data.material_manager - DEBUG - Reading materials with selection: 5, filters: {}


Feature1: 1
Feature4999: 4999


**Note**: Using external tables allows you to store and access large amounts of data efficiently without overloading the main database tables.

---

## Reading Materials

Retrieve materials from the database using the `read` method. You can apply selection criteria to filter the results.

### Example 1: Reading All Materials

In [13]:
# Read all materials
results = mgdb.db_manager.read()

# Display the results
for result in results:
    print(f"ID: {result.id}")
    print(f"Row: {result}")
    print(f"Data: {result.data}")
    print('-' * 50)

2024-09-19 14:33:36,904 - matgraphdb.data.material_manager - DEBUG - Reading materials with selection: None, filters: {}


ID: 1
Row: <AtomsRow: formula=Fe, keys=has_structure>
Data: {'density': 7.87}
--------------------------------------------------
ID: 2
Row: <AtomsRow: formula=TiO2, keys=has_structure>
Data: {}
--------------------------------------------------
ID: 3
Row: <AtomsRow: formula=Fe, keys=has_structure>
Data: {'density': 7.87, 'color': 'silver'}
--------------------------------------------------
ID: 4
Row: <AtomsRow: formula=Fe, keys=has_structure,is_molecule,phase>
Data: {}
--------------------------------------------------
ID: 5
Row: <AtomsRow: formula=Fe, keys=has_structure>
Data: {}
--------------------------------------------------


### Example 2: Reading Materials Without Structure

In [14]:
# Read materials where 'has_structure' is False
results = mgdb.db_manager.read(selection='has_structure=False')

# Display the results
for result in results:
    print(f"ID: {result.id}")
    print(f"Data: {result.data}")
    print('-' * 50)

2024-09-19 14:33:36,938 - matgraphdb.data.material_manager - DEBUG - Reading materials with selection: has_structure=False, filters: {}


ID: 2
Data: {}
--------------------------------------------------


---

## Updating Materials

Update existing materials using the `update_material` method by specifying the material ID and the new data.

### Example: Updating Material Data

In [15]:
# Update the density of material with ID 1
mgdb.db_manager.update_material(
    material_id=1,
    data={'density': 9.00}
)

# Read the updated material
results = mgdb.db_manager.read(selection=1)
for result in results:
    print(f"ID: {result.id}")
    print(f"Data: {result.data}")

2024-09-19 14:33:36,954 - matgraphdb.data.material_manager - INFO - Updating material with ID 1.
2024-09-19 14:33:36,955 - matgraphdb.data.material_manager - DEBUG - Updating properties: {'density': 9.0}
2024-09-19 14:33:36,956 - matgraphdb.data.material_manager - DEBUG - Opening database connection for updating.
2024-09-19 14:33:37,051 - matgraphdb.data.material_manager - INFO - Material with ID 1 updated successfully.
2024-09-19 14:33:37,052 - matgraphdb.data.material_manager - DEBUG - Reading materials with selection: 1, filters: {}


ID: 1
Data: {'density': 9.0}


---

## Deleting Materials

Remove materials from the database using the `delete_material` method.

### Example: Deleting a Material

In [16]:
# Delete material with ID 4
mgdb.db_manager.delete_material(material_ids=[4])

# Try to read the deleted material
results = mgdb.db_manager.read(selection=4)

results = mgdb.db_manager.read()
for result in results:
    print(result.id)
    print(result)
    print('-'*100)


2024-09-19 14:33:37,077 - matgraphdb.data.material_manager - INFO - Deleting material with ID [4].
2024-09-19 14:33:37,259 - matgraphdb.data.material_manager - INFO - Material with ID [4] deleted successfully.
2024-09-19 14:33:37,260 - matgraphdb.data.material_manager - DEBUG - Reading materials with selection: 4, filters: {}
2024-09-19 14:33:37,261 - matgraphdb.data.material_manager - DEBUG - Reading materials with selection: None, filters: {}


1
<AtomsRow: formula=Fe, keys=has_structure>
----------------------------------------------------------------------------------------------------
2
<AtomsRow: formula=TiO2, keys=has_structure>
----------------------------------------------------------------------------------------------------
3
<AtomsRow: formula=Fe, keys=has_structure>
----------------------------------------------------------------------------------------------------
5
<AtomsRow: formula=Fe, keys=has_structure>
----------------------------------------------------------------------------------------------------


---

## Conclusion

In this tutorial, we've covered the basics of managing a materials database using `MatGraphDB` and `MaterialDatabaseManager`. We've learned how to:

- Initialize the database
- Add materials with or without atomic structures
- Read and filter materials from the database
- Update material data
- Delete materials from the database

For more advanced features and functionalities, you can refer to the ASE Database Documentation and explore additional methods provided by the `MaterialDatabaseManager` class.

---

**Note:** This tutorial assumes that you have the `matgraphdb` package installed and properly configured in your Python environment.